In [1]:
import pandas as pd
import numpy as np
import string
import re
from collections import Counter

In [2]:
raw = pd.read_csv( "../data/interim/lazada_review_clean_1.csv")
slang_words = pd.read_csv("../data/external/kbba_ID.txt",
                        sep="\t", header=None)
slang = pd.read_csv("../data/external/slangword_ID.txt",
                        sep=":", header=None)
slang_words = pd.concat([slang_words, slang])

In [3]:
slang_words.drop_duplicates(inplace=True)
slang_words = dict(zip(slang_words[0],slang_words[1]))

In [4]:
print(raw.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19872 entries, 0 to 19871
Data columns (total 4 columns):
rating        19872 non-null int64
review        19872 non-null object
avg_word      19872 non-null float64
word_count    19872 non-null int64
dtypes: float64(1), int64(2), object(1)
memory usage: 621.1+ KB
None


In [5]:
raw['review'][0]

"['pengiriman', 'ninja', 'sangat', 'lama', 'jauh', 'berbeda', 'kurir', 'internal', 'lazada', 'baik', 'kasih', 'opsi', 'pelanggan', 'bisa', 'memilih', 'kurir']"

In [6]:
raw.head()

,rating,review,avg_word,word_count
0,1,"['pengiriman', 'ninja', 'sangat', 'lama', 'jau...",5.812500,16
1,1,"['pesananku', 'nomor', 'order', 'terkirim', 'b...",6.152985,268
2,5,"['tidak', 'sia', 'sia', 'susah', 'payah', 'iku...",6.105263,19
3,5,"['tujuh', 'kali', 'gagal', 'flashsale', 'padah...",6.125000,16
4,5,"['kurang', 'setuju', 'flash sale', 'sih', 'str...",6.678571,28


In [7]:
def csv_string_to_list(csv_string):
    return csv_string[1:-1].split(", ")

def string_without_quotes(word_list):
    new  = []
    for word in word_list:
        new.append(word.replace("'",""))
    return new
                   
raw['review'] = raw['review'].apply(csv_string_to_list)
raw['review'] = raw['review'].apply(string_without_quotes)

In [8]:
def mapping_slang_words(review):
    return [slang_words[word] if word in slang_words else word for word in review]
raw['review'] = raw['review'].apply(mapping_slang_words)

In [9]:
raw['review']

0        [pengiriman, ninja, sangat, lama, jauh, berbed...
1        [pesananku, nomor, order, terkirim, baik, nomo...
2        [tidak, sia, sia, susah, payah, ikutan, flashs...
3        [tujuh, kali, gagal, flashsale, padahal, nga, ...
4        [kurang, setuju, flash sale, sih, strategi, ma...
5        [kurir, ninja, express, lambat, kecewa, sih, l...
6        [pengiriman, kota, depok, membutuhkan, sebelas...
7        [barang, cuma, lama, lebih baik, pakai, eksped...
8        [saran, tolong, pengiriman, paket, jangan, jas...
9        [telepon genggam, xiaomi, oke, cepat, kurir, r...
10       [beli, flash, sale, xiaomi, redmi, powerbank, ...
11       [barangnya, cepat, tepat, sayang, pesan, baran...
12       [proses, pengiriman, pesanan, sangat, cepat, k...
13       [barang, bagus, garansi, tam, dikirim, jam, ba...
14       [dapat, redmi, plus, emas, hitam, pagi, redmi,...
15       [lazada, kadang, lama, mengiriminya, kadang, c...
16       [kali, flash, sale, barang, aman, rapi, pengir.

In [10]:
def remove_single_alphabet_only(review):
    return [word for word in review if word not in string.ascii_lowercase]

def remove_too_short_words(review):
    return [word for word in review if len(word) > 2]

In [11]:
raw['review'] = raw['review'].apply(remove_single_alphabet_only)
raw['review'] = raw['review'].apply(remove_too_short_words)

In [12]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19872 entries, 0 to 19871
Data columns (total 4 columns):
rating        19872 non-null int64
review        19872 non-null object
avg_word      19872 non-null float64
word_count    19872 non-null int64
dtypes: float64(1), int64(2), object(1)
memory usage: 621.1+ KB


In [13]:
raw = raw.dropna(subset=['review'],how='all')

In [14]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19872 entries, 0 to 19871
Data columns (total 4 columns):
rating        19872 non-null int64
review        19872 non-null object
avg_word      19872 non-null float64
word_count    19872 non-null int64
dtypes: float64(1), int64(2), object(1)
memory usage: 776.2+ KB


In [15]:
raw = raw.dropna(subset=['review'],how='all')
raw = raw[raw['review'].map(len) > 0]
def convert_list_to_string(word_list):
    return ",".join(word_list)
raw['review'] = raw['review'].apply(convert_list_to_string)
raw = raw.dropna(subset=['review'],how='all')
raw = raw[raw['review'].map(len) > 0]

In [16]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19838 entries, 0 to 19871
Data columns (total 4 columns):
rating        19838 non-null int64
review        19838 non-null object
avg_word      19838 non-null float64
word_count    19838 non-null int64
dtypes: float64(1), int64(2), object(1)
memory usage: 774.9+ KB


In [17]:
def standardize_thanks(review):
    return re.sub("terima,kasih","terima kasih", review)
raw['review'] = raw['review'].apply(standardize_thanks)

In [18]:
raw.to_csv("../data/interim/lazada_review_clean_2.csv", index=False)

In [19]:
most_common = pd.Series(''.join(raw['review']).split(",")).value_counts()[:10]
most_common

tidak           6272
barang          4917
bagus           4149
cepat           3668
lazada          3471
sesuai          3052
terima kasih    2991
pengiriman      2576
bisa            2162
harga           1919
dtype: int64

In [20]:
def add_coma(review):
    return review + ','
raw['review'] = raw['review'].apply(add_coma)

In [21]:
least_common = Counter("".join(raw["review"]).split(",")).most_common()[-10:]
least_common

[('jayadelapan', 1),
 ('selaras', 1),
 ('resletingnya', 1),
 ('pnirimn', 1),
 ('kuningan', 1),
 ('meetingnya', 1),
 ('cardholder', 1),
 ('garuda', 1),
 ('ckckckck', 1),
 ('', 1)]